# Expression Templates

## Motivation

To implement mathematical operators such as the addition (`+`), while respecting the notations and expectations of mathematicians, we recommended that these operators build and return a new value, without modifying the operands. If you add up huge matrices, the consequences on the calculation time can be dramatic, especially because of all the intermediary values which are produced before the final result is reached:

In [ ]:
using BigMatrix = SquareMatrix<double, 10000> ;
BigMatrix m1, m2, m3, m4, m5 ;               // create matrices and
// ...                                       // give them values
BigMatrix result = m1 + m2 + m3 + m4 + m5 ;  // compute their addition

Classical programming techniques will lead to the creation of four temporary matrices (one for each call for `+`), and the course of a sequence of four loops through all the elements of the matrix. With the idiom known as "expression templates", it is sometimes possible to eliminate temporary matrices and to merge the loops, without changing the syntax of the client code above. The resulting code uses less memory and runs
incredibly faster.

## Example

For the purposes of the demonstration, we will be using vectors of 3 values (but they could also contain 1000+ values), and we focus on a simple expression based on + and - :

In [ ]:
%%file tmp.vector.h

#include <iostream>

struct Vector
 { double data[3] ; } ;
    
Vector operator+( Vector const & v1, Vector const & v2 )
 { return Vector{v1.data[0]+v2.data[0],v1.data[1]+v2.data[1],v1.data[2]+v2.data[2]} ; }

Vector operator-( Vector const & v1, Vector const & v2 )
 { return Vector{v1.data[0]-v2.data[0],v1.data[1]-v2.data[1],v1.data[2]-v2.data[2]} ; }
    
std::ostream & operator<<( std::ostream & os, Vector const & v )
 { return (os<<v.data[0]<<"|"<<v.data[1]<<"|"<<v.data[2]) ; }

In [ ]:
%%file tmp.expressions-templates.cpp

#include "tmp.vector.h"

int main()
{
  Vector v1{1,2,3}, v2{0.1,0.2,0.3}, v3{0.01,0.02,0.03}, v4{0,0,0} ;
  v4 = v1+(v2-v3) ;
  std::cout<<v4<<std::endl ;
  return 0 ;
} 

In [ ]:
!rm -f tmp.expressions-templates.exe && g++ -std=c++17 tmp.expressions-templates.cpp -o tmp.expressions-templates.exe

In [ ]:
!./tmp.expressions-templates.exe

The goal is to replace the naive implementation of the + and - operators, between instances of Vector, which leads to three loops through the list of values and the building of two temporary objects. Instead, we want the expression `v1+(v2-v3)` to return a structure that is as basic as possible, without any calculation or loops, the latter being realised in one go at the last moment, at the final assignment to `v4`. This can be described as *lazy evaluation*.

## Elements of the intermediate structure

Our intermediate structure will be built like a tree of objects representing the operations, the operands constituting the expression. Each element of the structure must behave as an iterator: being able to return its current value and to increment itself.

In [ ]:
%%file tmp.nodes.h

// Addition
template <typename Itr1, typename Itr2>
struct NodeAdd
 {
  Itr1 itr1 ; Itr2 itr2 ;
  void operator ++() { ++itr1 ; ++itr2 ; }
  double operator *() const { return (*itr1)+(*itr2) ; }
 } ;

// Soustraction 
template <typename Itr1, typename Itr2>
struct NodeSub
 {
  Itr1 itr1 ; Itr2 itr2 ;
  void operator ++() { ++itr1 ; ++itr2 ; }
  double operator *() const { return (*itr1)-(*itr2) ; }
 } ;

Below, we build a very simple structure, which is the sum of the elements of two tables.

In [ ]:
%%file tmp.expressions-templates.cpp

#include <iostream>
#include "tmp.nodes.h"

int main()
 {
  int array1[5] = { 1, 2, 3, 4, 5 } ;
  double array2[5] = { .1, .2, .3, .4, .5 } ;

  NodeAdd<int *,double *> add {array1,array2} ;
  for ( int i = 0 ; i<5 ; ++i )
   { std::cout<<(*add)<<" " ; ++add ; }
 }

In [ ]:
!rm -f tmp.expressions-templates.exe && g++ -std=c++17 tmp.expressions-templates.cpp -o tmp.expressions-templates.exe

In [ ]:
!./tmp.expressions-templates.exe

### Exercice

Modify the code above, so to test different variations :
* a substraction;
* **the sum of three tables**; 
* replacing the addresses pointing to the beginning of the tables by a call to `std::begin`, which works with either the raw tables or the other collections of the standard library;
* replacing the array with `std::vector`;
* factoring the operator `++` and the `Node` variables into a `NodeBase` base class.

## Construction of the intermediate structure for Vector

Let's take our 'Vector' class and provide the methods `begin` and `end` which allow to build structures, as we
did previously. Below, this leads to this new variant of `Vector`, and an implementation of operators that builds a `Node` structure.

In [ ]:
%%file tmp.vector.h

#include <iostream>
#include "tmp.nodes.h"

struct Vector
 {
  double data[3] ;
  // assignment
  //...   
 } ;

std::ostream & operator<<( std::ostream & os, Vector const & v ) 
 { return (os<<v.data[0]<<"|"<<v.data[1]<<"|"<<v.data[2]) ; }

#include "tmp.vector-add.h"
#include "tmp.vector-sub.h"

In [ ]:
%%file tmp.vector-add.h

template <class Node1, class Node2>
auto operator +(const Node1 & n1, const Node2 & n2)
 { return NodeAdd<Node1,Node2>{n1,n2} ; }

template <class Node>
auto operator +(Vector const & v, Node const & n)
 { return NodeAdd<double const *,Node>{v.data,n} ; }

template <class Node>
auto operator +(Node const & n, Vector const & v)
 { return NodeAdd<Node,double const *>{n,v.data} ; } 

auto operator +(Vector const & v1, Vector const & v2)
 { return NodeAdd<double const *,double const *>{v1.data,v2.data} ; }

In [ ]:
%%file tmp.vector-sub.h

template <class Node1, class Node2>
auto operator -(const Node1 & n1, const Node2 & n2)
 { return NodeSub<Node1,Node2>{n1,n2} ; }

template <class Node>
auto operator -(Vector const & v, Node const & n)
 { return NodeSub<double const *,Node>{v.data,n} ; }

template <class Node>
auto operator -(Node const & n, Vector const & v)
 { return NodeSub<Node,double const *>{n,v.data} ; } 

auto operator -(Vector const & v1, Vector const & v2)
 { return NodeSub<double const *,double const *>{v1.data,v2.data} ; }

### Exercice

Below, we use the initial code of this course, and we would like it to compile and display the right result. Will you be able to add the missing assignment operator to the `tmp.vector.h` file above ?

In [ ]:
%%file tmp.expressions-templates.cpp

#include "tmp.vector.h"

int main()
 {
  Vector v1{1,2,3}, v2{0.1,0.2,0.3}, v3{0.01,0.02,0.03}, v4{0,0,0} ;
  v4 = v1+(v2-v3) ;
  std::cout<<v4<<std::endl ;
  return 0 ;
 }

In [ ]:
!rm -f tmp.expressions-templates.exe && g++ -std=c++17 tmp.expressions-templates.cpp -o tmp.expressions-templates.exe

In [ ]:
!./tmp.expressions-templates.exe

© *CNRS 2021*  
*This document was created by David Chamont and translated by Patricia Mary. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*